In [1]:
import pandas as pd
import sweetviz as sv
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import surprise

c:\Users\marioestevao\anaconda3\envs\venvMAL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_animes = pd.read_csv('./raw/animes.csv')
df_animes.head(2)

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...


In [3]:
df_reviews = pd.read_csv('./raw/reviews.csv')
df_reviews.head()

,uid,profile,anime_uid,text,score,scores,link
0,255938,DesolatePsyche,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=255938
1,259117,baekbeans,34599,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=259117
2,253664,skrn,28891,\n \n \n \n ...,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=253664
3,8254,edgewalker00,2904,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '9...",https://myanimelist.net/reviews.php?id=8254
4,291149,aManOfCulture99,4181,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=291149


In [4]:
df_users = pd.read_csv('./raw/profiles.csv')
df_users.head()

,profile,gender,birthday,favorites_anime,link
0,DesolatePsyche,Male,"Oct 2, 1994","['33352', '25013', '5530', '33674', '1482', '2...",https://myanimelist.net/profile/DesolatePsyche
1,baekbeans,Female,"Nov 10, 2000","['11061', '31964', '853', '20583', '918', '925...",https://myanimelist.net/profile/baekbeans
2,skrn,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32...",https://myanimelist.net/profile/skrn
3,edgewalker00,Male,Sep 5,"['5680', '849', '2904', '3588', '37349']",https://myanimelist.net/profile/edgewalker00
4,aManOfCulture99,Male,"Oct 30, 1999","['4181', '7791', '9617', '5680', '2167', '4382...",https://myanimelist.net/profile/aManOfCulture99


### METADATA EDA

#### Dataset Animes

In [5]:
df_animes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19311 entries, 0 to 19310
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   uid         19311 non-null  int64  
 1   title       19311 non-null  object 
 2   synopsis    18336 non-null  object 
 3   genre       19311 non-null  object 
 4   aired       19311 non-null  object 
 5   episodes    18605 non-null  float64
 6   members     19311 non-null  int64  
 7   popularity  19311 non-null  int64  
 8   ranked      16099 non-null  float64
 9   score       18732 non-null  float64
 10  img_url     19131 non-null  object 
 11  link        19311 non-null  object 
dtypes: float64(3), int64(3), object(6)
memory usage: 1.8+ MB


In [6]:
#df_animes['synopsis'] = df_animes['synopsis'].fillna('No information')

Checando duplicados

In [7]:
df_animes.shape

(19311, 12)

In [8]:
title_counts = df_animes['uid'].value_counts()
df_animes[df_animes['uid'].isin(title_counts[title_counts > 1].index)]

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...
...,...,...,...,...,...,...,...,...,...,...,...,...
19306,32979,Flip Flappers,Cocona is an average middle schooler living wi...,"['Sci-Fi', 'Adventure', 'Comedy', 'Magic']","Oct 6, 2016 to Dec 29, 2016",13.0,134252,843,1070.0,7.73,https://cdn.myanimelist.net/images/anime/4/822...,https://myanimelist.net/anime/32979/Flip_Flappers
19307,123,Fushigi Yuugi,"While visiting the National Library, junior-hi...","['Adventure', 'Fantasy', 'Magic', 'Martial Art...","Apr 6, 1995 to Mar 28, 1996",52.0,84407,1292,1071.0,7.73,https://cdn.myanimelist.net/images/anime/2/201...,https://myanimelist.net/anime/123/Fushigi_Yuugi
19308,1281,Gakkou no Kaidan,"Years ago, all of the ghosts in a haunted scho...","['Mystery', 'Horror', 'Supernatural']","Oct 22, 2000 to Mar 25, 2001",19.0,83093,1314,1073.0,7.73,https://cdn.myanimelist.net/images/anime/9/183...,https://myanimelist.net/anime/1281/Gakkou_no_K...
19309,450,InuYasha Movie 2: Kagami no Naka no Mugenjo,Inuyasha and company have finally destroyed Na...,"['Action', 'Adventure', 'Comedy', 'Historical'...","Dec 21, 2002",1.0,71989,1469,1077.0,7.73,https://cdn.myanimelist.net/images/anime/1162/...,https://myanimelist.net/anime/450/InuYasha_Mov...


In [9]:
df_animes.loc[df_animes['uid'] == 450]

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
1095,450,InuYasha Movie 2: Kagami no Naka no Mugenjo,Inuyasha and company have finally destroyed Na...,"['Action', 'Adventure', 'Comedy', 'Historical'...","Dec 21, 2002",1.0,71989,1469,1077.0,7.73,https://cdn.myanimelist.net/images/anime/1162/...,https://myanimelist.net/anime/450/InuYasha_Mov...
7739,450,InuYasha Movie 2: Kagami no Naka no Mugenjo,Inuyasha and company have finally destroyed Na...,"['Action', 'Adventure', 'Comedy', 'Historical'...","Dec 21, 2002",1.0,71972,1469,1077.0,7.73,https://cdn.myanimelist.net/images/anime/1162/...,https://myanimelist.net/anime/450/InuYasha_Mov...
19309,450,InuYasha Movie 2: Kagami no Naka no Mugenjo,Inuyasha and company have finally destroyed Na...,"['Action', 'Adventure', 'Comedy', 'Historical'...","Dec 21, 2002",1.0,71989,1469,1077.0,7.73,https://cdn.myanimelist.net/images/anime/1162/...,https://myanimelist.net/anime/450/InuYasha_Mov...


In [10]:
df_animes = df_animes.drop_duplicates(subset=['uid'], keep='first').sort_values('uid')

In [11]:
df_animes.shape

(16216, 12)

In [12]:
df_animes.rename(columns={'uid': 'anime_id',
                          'genre': 'anime_genre',
                          'ranked': 'rank',
                          'link': 'url'}, inplace=True)

#### Dataset Users

In [13]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81727 entries, 0 to 81726
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   profile          81727 non-null  object
 1   gender           53856 non-null  object
 2   birthday         46807 non-null  object
 3   favorites_anime  81727 non-null  object
 4   link             81727 non-null  object
dtypes: object(5)
memory usage: 3.1+ MB


Apenas informações sensívels têm valor nulo, então tudo ok

In [14]:
df_users.shape

(81727, 5)

Checando duplicados

In [15]:
name_counts = df_users['profile'].value_counts()
df_users[df_users['profile'].isin(name_counts[name_counts > 1].index)]

,profile,gender,birthday,favorites_anime,link
0,DesolatePsyche,Male,"Oct 2, 1994","['33352', '25013', '5530', '33674', '1482', '2...",https://myanimelist.net/profile/DesolatePsyche
1,baekbeans,Female,"Nov 10, 2000","['11061', '31964', '853', '20583', '918', '925...",https://myanimelist.net/profile/baekbeans
2,skrn,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32...",https://myanimelist.net/profile/skrn
3,edgewalker00,Male,Sep 5,"['5680', '849', '2904', '3588', '37349']",https://myanimelist.net/profile/edgewalker00
4,aManOfCulture99,Male,"Oct 30, 1999","['4181', '7791', '9617', '5680', '2167', '4382...",https://myanimelist.net/profile/aManOfCulture99
...,...,...,...,...,...
81722,lovelessxd,Female,"Aug 6, 1992","['853', '5114']",https://myanimelist.net/profile/lovelessxd
81723,Shattered_Angel,Female,"Sep 6, 1994","['150', '27', '1520', '121', '31452', '32995',...",https://myanimelist.net/profile/Shattered_Angel
81724,FluffyWalrus,Male,NaN,"['121', '43', '237', '202', '205']",https://myanimelist.net/profile/FluffyWalrus
81725,camco,Female,Sep 23,"['199', '4224', '7054', '13601', '14713']",https://myanimelist.net/profile/camco


In [16]:
df_users.loc[df_users['profile'] == 'FluffyWalrus']

,profile,gender,birthday,favorites_anime,link
29897,FluffyWalrus,Male,NaN,"['121', '43', '237', '202', '205']",https://myanimelist.net/profile/FluffyWalrus
56136,FluffyWalrus,Male,NaN,"['121', '43', '237', '202', '205']",https://myanimelist.net/profile/FluffyWalrus
81724,FluffyWalrus,Male,NaN,"['121', '43', '237', '202', '205']",https://myanimelist.net/profile/FluffyWalrus


In [17]:
df_users[df_users.duplicated()]

,profile,gender,birthday,favorites_anime,link
25792,Chappie7815,Male,"Jun 29, 2000","['36904', '18679']",https://myanimelist.net/profile/Chappie7815
25793,VoidHawk,Male,"Jan 15, 1971","['195', '318', '790', '2217', '1249', '145', '...",https://myanimelist.net/profile/VoidHawk
25794,Eoussama,Male,"Jun 27, 1998","['37510', '4181', '21', '2035']",https://myanimelist.net/profile/Eoussama
25795,ReLCreed,Male,NaN,"['790', '16498', '2966', '22319', '13759']",https://myanimelist.net/profile/ReLCreed
25796,0Seru0,Male,"Dec 18, 1990","['7661', '582', '33', '2904', '1425', '1453', ...",https://myanimelist.net/profile/0Seru0
...,...,...,...,...,...
81722,lovelessxd,Female,"Aug 6, 1992","['853', '5114']",https://myanimelist.net/profile/lovelessxd
81723,Shattered_Angel,Female,"Sep 6, 1994","['150', '27', '1520', '121', '31452', '32995',...",https://myanimelist.net/profile/Shattered_Angel
81724,FluffyWalrus,Male,NaN,"['121', '43', '237', '202', '205']",https://myanimelist.net/profile/FluffyWalrus
81725,camco,Female,Sep 23,"['199', '4224', '7054', '13601', '14713']",https://myanimelist.net/profile/camco


Para decidir o método de exclusão dos duplicados aqui, precisamos saber se há alguma alteração ou informação entre as entradas duplicadas de usuário

In [18]:
# Identifique todas as linhas duplicadas com base na coluna 'column1'
duplicates1 = df_users[df_users.duplicated(subset='profile', keep=False)]

# Filtrar as linhas que são duplicatas em todas as colunas
test = duplicates1[~duplicates1.duplicated(keep=False)]

# Agora, 'filtered_duplicates' contém as linhas que são duplicadas com base em 'column1', 
# mas têm valores diferentes em algumas outras colunas

test.loc[test['profile'] == 'FluffyWalrus']

,profile,gender,birthday,favorites_anime,link


In [19]:
test

,profile,gender,birthday,favorites_anime,link
11086,xre,NaN,NaN,"['33502', '37450', '5081', '31240', '30831', '...",https://myanimelist.net/profile/xre
15514,MrHaibo,Male,NaN,"['33926', '30276', '44', '37450', '32281', '50...",https://myanimelist.net/profile/MrHaibo
17959,Wumbo_Combo,NaN,"Sep 20, 1998","['35247', '11665', '801', '35180', '40767', '6...",https://myanimelist.net/profile/Wumbo_Combo
24106,SalmanAlfarisi21,NaN,NaN,"['10110', '4181']",https://myanimelist.net/profile/SalmanAlfarisi21
25066,treas0n,Male,Jun 2,"['32281', '31953', '37976', '31933', '37396']",https://myanimelist.net/profile/treas0n
25887,NishizawaMihashi,Male,1995,"['2025', '516', '5114', '245', '28223', '35413']",https://myanimelist.net/profile/NishizawaMihashi
26750,-HippySnob-,Male,"May 21, 1999",[],https://myanimelist.net/profile/-HippySnob-
27002,mpdissonance,Male,"Aug 22, 1995","['5114', '36649', '43', '457', '32281', '19', ...",https://myanimelist.net/profile/mpdissonance
32312,Kuru-Hime,Female,Feb 4,"['9253', '934', '38000', '4181', '33486', '654...",https://myanimelist.net/profile/Kuru-Hime
32744,Wumbo_Combo,NaN,"Sep 20, 1998","['35247', '11665', '801', '35180', '6594', '35...",https://myanimelist.net/profile/Wumbo_Combo


Eram poucos dados, então eu os chequei manualmente. Usando o usuario treas0n como exemplo, podemos ver que ele tem 2 perfis, mas a entrada com index menor tem mais informações, então eu removi a entrada com index maior.

In [20]:
df_users = df_users.drop_duplicates(subset=['profile'], keep='first').sort_values('profile')
df_users.shape

(47885, 5)

In [21]:
df_users.loc[df_users['profile'] == 'treas0n']

,profile,gender,birthday,favorites_anime,link
25066,treas0n,Male,Jun 2,"['32281', '31953', '37976', '31933', '37396']",https://myanimelist.net/profile/treas0n


In [22]:
df_users.rename(columns={'profile': 'user_name',
                         'gender': 'user_gender',
                         'link': 'user_url'}, inplace=True)

In [23]:
df_users

,user_name,user_gender,birthday,favorites_anime,user_url
10861,-----noname-----,NaN,"Dec 31, 2019","['6774', '245', '2001', '11061', '16592', '157...",https://myanimelist.net/profile/-----noname-----
20640,---SnowFlake---,NaN,NaN,"['2904', '6773', '10790']",https://myanimelist.net/profile/---SnowFlake---
66586,---was-----,NaN,NaN,[],https://myanimelist.net/profile/---was-----
18169,--EYEPATCH--,Male,"Oct 28, 2000",[],https://myanimelist.net/profile/--EYEPATCH--
6207,--Mizu--,Female,"Jul 3, 1995","['21', '177', '6864', '4081', '5678', '23289']",https://myanimelist.net/profile/--Mizu--
...,...,...,...,...,...
17642,zzyamuraihazz,Male,"Oct 24, 1995",[],https://myanimelist.net/profile/zzyamuraihazz
63584,zzz-anime,Male,NaN,[],https://myanimelist.net/profile/zzz-anime
56449,zzzb,Male,Jun 30,['9253'],https://myanimelist.net/profile/zzzb
9995,zzzjynne,NaN,Feb 22,"['31149', '1575', '2904', '43', '5337', '790',...",https://myanimelist.net/profile/zzzjynne


#### Dataset Reviews

In [24]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192112 entries, 0 to 192111
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   uid        192112 non-null  int64 
 1   profile    192112 non-null  object
 2   anime_uid  192112 non-null  int64 
 3   text       192112 non-null  object
 4   score      192112 non-null  int64 
 5   scores     192112 non-null  object
 6   link       192112 non-null  object
dtypes: int64(3), object(4)
memory usage: 10.3+ MB


Aqui não tem registro nulo então seguiremos

In [25]:
df_reviews.shape

(192112, 7)

In [26]:
review_counts = df_reviews['uid'].value_counts()
df_reviews[df_reviews['uid'].isin(review_counts[review_counts > 1].index)]

,uid,profile,anime_uid,text,score,scores,link
0,255938,DesolatePsyche,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=255938
1,259117,baekbeans,34599,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=259117
2,253664,skrn,28891,\n \n \n \n ...,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=253664
3,8254,edgewalker00,2904,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '9...",https://myanimelist.net/reviews.php?id=8254
4,291149,aManOfCulture99,4181,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=291149
...,...,...,...,...,...,...,...
192107,240067,Unicorn819,1281,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '5', 'Animation': '1...",https://myanimelist.net/reviews.php?id=240067
192108,285777,ShizzoSVH,1281,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=285777
192109,286904,AlluMan96,1281,\n \n \n \n ...,3,"{'Overall': '3', 'Story': '3', 'Animation': '1...",https://myanimelist.net/reviews.php?id=286904
192110,287903,AgentK300,1281,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '3', 'Animation': '...",https://myanimelist.net/reviews.php?id=287903


In [27]:
df_reviews.loc[df_reviews['uid'] == 255938]

,uid,profile,anime_uid,text,score,scores,link
0,255938,DesolatePsyche,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=255938
60897,255938,DesolatePsyche,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=255938


In [28]:
df_reviews[df_reviews.duplicated()]

,uid,profile,anime_uid,text,score,scores,link
54267,321837,shoxs84,40269,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '10', 'Animation': '...",https://myanimelist.net/reviews.php?id=321837
54268,321498,Hei316,40269,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '7', 'Animation': '8...",https://myanimelist.net/reviews.php?id=321498
54269,322786,Ayanamihhh,40269,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=322786
54270,321183,Zaky1435,40269,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '8...",https://myanimelist.net/reviews.php?id=321183
54271,321148,YeeYeeAss,40269,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '0', 'Animation': '...",https://myanimelist.net/reviews.php?id=321148
...,...,...,...,...,...,...,...
192107,240067,Unicorn819,1281,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '5', 'Animation': '1...",https://myanimelist.net/reviews.php?id=240067
192108,285777,ShizzoSVH,1281,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=285777
192109,286904,AlluMan96,1281,\n \n \n \n ...,3,"{'Overall': '3', 'Story': '3', 'Animation': '1...",https://myanimelist.net/reviews.php?id=286904
192110,287903,AgentK300,1281,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '3', 'Animation': '...",https://myanimelist.net/reviews.php?id=287903


O número de toda as colunas duplicadas é diferente das duplicadas apenas pelo uid, então vamos fazer o mesmo procedimento do dataset users

In [29]:
# Identifique todas as linhas duplicadas com base na coluna 'column1'
duplicates2 = df_reviews[df_reviews.duplicated(subset='uid', keep='first')]

# Filtrar as linhas que são duplicatas em todas as colunas
test2 = duplicates2[~duplicates2.duplicated(keep=False)].sort_values('uid', ascending=True)

# Agora, 'filtered_duplicates' contém as linhas que são duplicadas com base em 'column1', 
# mas têm valores diferentes em algumas outras colunas

test2.loc[test2['uid'] == 321148]

,uid,profile,anime_uid,text,score,scores,link


In [30]:
df_reviews = df_reviews.drop_duplicates(subset=['uid'], keep='first').sort_values('uid', ascending=True)
df_reviews.shape

(130519, 7)

In [31]:
df_reviews.columns

Index(['uid', 'profile', 'anime_uid', 'text', 'score', 'scores', 'link'], dtype='object')

In [32]:
df_reviews.rename(columns={'uid': 'review_id',
                           'profile': 'review_user_name',
                           'score': 'review_score',
                           'scores': 'review_scores',
                           'link': 'review_url'}, inplace=True)

#### Criando novos csvs com os arquivos limpos

In [33]:
print(df_animes.shape)
print(df_users.shape)
print(df_reviews.shape)

(16216, 12)
(47885, 5)
(130519, 7)


In [34]:
#df_animes.to_csv('animes_cleaned.csv', index=False)
#df_users.to_csv('users_cleaned.csv', index=False)
#df_reviews.to_csv('reviews_cleaned.csv', index=False)

### EDA

In [35]:
animes = pd.read_csv('./silver/animes_cleaned.csv')
users = pd.read_csv('./silver/users_cleaned.csv')
reviews = pd.read_csv('./silver/reviews_cleaned.csv')

Animes

In [36]:
dash_animes = sv.analyze(animes)

[Summarizing dataframe]                      |          | [  0%]   00:00 -> (? left)

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)


In [37]:
dash_animes.show_html()

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


Insights Iniciais do dataset animes:

- Há uma forte correlação positiva entre **popularity** e **rank**, o que mostra como animes bem avaliados têm uma tendência a serem mais assistidos
- Há uma forte correlação negativa entre **score** e **ranqueamento**, afinal, quanto maior o score, menor o número no ranking
- Há uma forte correlação negativa entre **members** e **popularity**, afinal, quanto maior o número de membros, maior a classificação em popularidade
- Há uma forte correlação positiva entre **members** e **score**

Dicionário das colunas:

- ***rank:*** ranking baseado na pontuação geral do anime em comparação aos outros
- ***popularity:*** ranking baseado na quantidade de pessoas que adicionaram o anime à lista pessoal
- ***score:*** pontuação do anime baseado em reviews dos usuários
- ***members:*** quantidade de pessoas que adicionaram o anime à própria lista

Users

In [38]:
dash_users = sv.analyze(users)

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)


In [39]:
dash_users.show_html()

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


Insights Iniciais do dataset users:

Há 3 valores principais na coluna **user_gender**, e a distribuição deles está assim:
- 69% Male
- 30% Female
- 1% Non-binary

Esses são valores relativos ao total de respostas válidas.

Não preenchidas: 36%

Reviews

In [40]:
dash_reviews = sv.analyze(reviews)

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)


In [41]:
dash_reviews.show_html()

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


Insights iniciais do dataset reviews:

A coluna **review_score** contém uma distribuição ascendente, com média de 7.3. Porém, algo estranho está nesses dados. Foi detectado um valor com review_score de **11!**

In [42]:
# Defina a largura máxima da coluna para None para exibir todo o texto
pd.set_option('display.max_colwidth', None)

In [43]:
reviews.loc[reviews['review_score'] == 11]

,review_id,review_user_name,anime_uid,text,review_score,review_scores,review_url
88391,252847,LeBrains,413,"\n \n \n \n \n \n \n \n more pics \n \n \n \n \n \n Overall \n 11 \n \n \n Story \n 11 \n \n \n Animation \n 11 \n \n \n Sound \n 11 \n \n \n Character \n 11 \n \n \n Enjoyment \n 11 \n \n \n \n\n \n A contemporanean masterpiece. The looks, the sound, everything is perfect with this animation. It should be exposed in museums for all the people i this world, such a perfection in the anime industry. \r\n \r\nThe story and the interation with its characters : \r\n The most well constructed characters i ever seen, in the well constructed world i have ever seen in a entertaiment media. A whole journey is seen in this most well done 19 minutes. But even with this perfection, this is'int a story for kids, the violence and cruelty seen in this show is unbelievable, you can feel the characters through they journey. It has the best ending that you will ever see. \r\n \r\nThe art : \r\n The most beatiful animation that you will ever see, it's like the real world, it can express the feelings of they animators fully with joy doing this masterpiece. \r\n \r\nThe soundtrack : \r\n I can still hear this overwhelming pressure being heard in the action scenes. The volume bursting in the calm scene only gives us the tension that is approaching. \r\n \r\nThe enjoyment: \r\n How can't i enjoy this show? It's just perfect! \r\n \r\n11/10 in an overall, it simply can' t have a score below this!\n\n \n \n Helpful \n \n",11,"{'Overall': '11', 'Story': '11', 'Animation': '11', 'Sound': '11', 'Character': '11', 'Enjoyment': '11'}",https://myanimelist.net/reviews.php?id=252847


Vamos checar que anime é esse

In [44]:
animes.loc[animes['anime_id'] == 413]

,anime_id,title,synopsis,anime_genre,aired,episodes,members,popularity,rank,score,img_url,url
384,413,Hametsu no Mars,"Several months after a probe returning from Mars burns up during reentry, strange creatures dubbed ""Ancients"" begin to appear throughout Tokyo. Aggressive and dangerous, they cannot be killed with ordinary weapons. As scientists struggle to find the cause of their sudden appearance, the monsters wreak havoc across the world. \r\n \r\nHumanity finds hope in the form of the MARS suit—a new weapon developed to take down these enemies. The suit, however, can only be worn by those with specific DNA. Enter Takeru Hinata, an ordinary teen with a troubled past and one of the few capable of piloting the only weapon against the Ancients. With the help of the AAST, a special police force established to defeat the dangerous creatures, the young man battles against demons both personal and global whilst trying to discover the true nature of Earth's new aggressors. \r\n \r\n[Written by MAL Rewrite]","['Sci-Fi', 'Horror']","Jul 6, 2005",1.0,40946,2153,11481.0,2.33,https://cdn.myanimelist.net/images/anime/7/68855.jpg,https://myanimelist.net/anime/413/Hametsu_no_Mars


### Preprocessamento

In [45]:
# Redefinir a largura máxima da coluna para o valor padrão
pd.reset_option('display.max_colwidth')

In [46]:
animes.columns


Index(['anime_id', 'title', 'synopsis', 'anime_genre', 'aired', 'episodes',
       'members', 'popularity', 'rank', 'score', 'img_url', 'url'],
      dtype='object')

In [47]:
users.columns

Index(['user_name', 'user_gender', 'birthday', 'favorites_anime', 'user_url'], dtype='object')

In [48]:
reviews.columns

Index(['review_id', 'review_user_name', 'anime_uid', 'text', 'review_score',
       'review_scores', 'review_url'],
      dtype='object')

In [49]:
# Criar um dataframe de usuários com informações de avaliações e animes favoritos
user_ratings = pd.merge(reviews, users, left_on='review_user_name', right_on='user_name', how='inner')
user_animes = pd.merge(user_ratings, animes, left_on='anime_uid', right_on='anime_id', how='inner')

# Agregar informações para criar um dataframe de usuário com informações consolidadas
user_data = user_animes.groupby('review_user_name').agg({
    'anime_uid': 'count',
    'anime_genre': lambda x: ', '.join(x.unique())  # Concatenar gêneros únicos
}).reset_index()

# Renomear colunas
user_data.columns = ['user_name', 'num_reviews', 'favorite_genres']

# Juntar as informações de usuário com as informações de animes
anime_data = pd.merge(reviews, animes, left_on='anime_uid', right_on='anime_id', how='inner')
anime_data = pd.merge(anime_data, user_data, left_on='review_user_name', right_on='user_name', how='left').drop(columns=['review_user_name'])


In [50]:
anime_data.loc[anime_data['anime_id'] == 1]

,review_id,anime_uid,text,review_score,review_scores,review_url,anime_id,title,synopsis,anime_genre,...,episodes,members,popularity,rank,score,img_url,url,user_name,num_reviews,favorite_genres
0,1,1,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '8', 'Animation': '...",https://myanimelist.net/reviews.php?id=1,1,Cowboy Bebop,"In the year 2071, humanity has colonized sever...","['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",...,26.0,930311,39,26.0,8.81,https://cdn.myanimelist.net/images/anime/4/196...,https://myanimelist.net/anime/1/Cowboy_Bebop,Xinil,14,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc..."
1,86,1,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '1...",https://myanimelist.net/reviews.php?id=86,1,Cowboy Bebop,"In the year 2071, humanity has colonized sever...","['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",...,26.0,930311,39,26.0,8.81,https://cdn.myanimelist.net/images/anime/4/196...,https://myanimelist.net/anime/1/Cowboy_Bebop,ivorykeys,6,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc..."
2,277,1,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=277,1,Cowboy Bebop,"In the year 2071, humanity has colonized sever...","['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",...,26.0,930311,39,26.0,8.81,https://cdn.myanimelist.net/images/anime/4/196...,https://myanimelist.net/anime/1/Cowboy_Bebop,the_pillows,2,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc..."
3,343,1,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=343,1,Cowboy Bebop,"In the year 2071, humanity has colonized sever...","['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",...,26.0,930311,39,26.0,8.81,https://cdn.myanimelist.net/images/anime/4/196...,https://myanimelist.net/anime/1/Cowboy_Bebop,jupiterjazz,3,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc..."
4,432,1,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '8...",https://myanimelist.net/reviews.php?id=432,1,Cowboy Bebop,"In the year 2071, humanity has colonized sever...","['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",...,26.0,930311,39,26.0,8.81,https://cdn.myanimelist.net/images/anime/4/196...,https://myanimelist.net/anime/1/Cowboy_Bebop,loneseeker1990,2,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,320955,1,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '1...",https://myanimelist.net/reviews.php?id=320955,1,Cowboy Bebop,"In the year 2071, humanity has colonized sever...","['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",...,26.0,930311,39,26.0,8.81,https://cdn.myanimelist.net/images/anime/4/196...,https://myanimelist.net/anime/1/Cowboy_Bebop,Norris_Packard,3,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc..."
357,321711,1,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '9...",https://myanimelist.net/reviews.php?id=321711,1,Cowboy Bebop,"In the year 2071, humanity has colonized sever...","['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",...,26.0,930311,39,26.0,8.81,https://cdn.myanimelist.net/images/anime/4/196...,https://myanimelist.net/anime/1/Cowboy_Bebop,Sidketsu,2,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc..."
358,323478,1,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '9', 'Animation': '...",https://myanimelist.net/reviews.php?id=323478,1,Cowboy Bebop,"In the year 2071, humanity has colonized sever...","['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",...,26.0,930311,39,26.0,8.81,https://cdn.myanimelist.net/images/anime/4/196...,https://myanimelist.net/anime/1/Cowboy_Bebop,midjune,1,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc..."
359,323933,1,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=323933,1,Cowboy Bebop,"In the year 2071, humanity has colonized sever...","['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",...,26.0,930311,3

In [51]:
user_animes.loc[user_animes['user_name'] == 'treas0n']

,review_id,review_user_name,anime_uid,text,review_score,review_scores,review_url,user_name,user_gender,birthday,...,synopsis,anime_genre,aired,episodes,members,popularity,rank,score,img_url,url
61651,300969,treas0n,30276,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '10', 'Animation': '...",https://myanimelist.net/reviews.php?id=300969,treas0n,Male,Jun 2,...,The seemingly ordinary and unimpressive Saitam...,"['Action', 'Sci-Fi', 'Comedy', 'Parody', 'Supe...","Oct 5, 2015 to Dec 21, 2015",12.0,1475219,5,50.0,8.68,https://cdn.myanimelist.net/images/anime/12/76...,https://myanimelist.net/anime/30276/One_Punch_Man
109675,313651,treas0n,34134,\n \n \n \n ...,5,"{'Overall': '5', 'Story': '7', 'Animation': '6...",https://myanimelist.net/reviews.php?id=313651,treas0n,Male,Jun 2,...,In the wake of defeating Boros and his mighty ...,"['Action', 'Comedy', 'Parody', 'Sci-Fi', 'Sein...","Apr 10, 2019 to Jul 3, 2019",12.0,607876,92,1732.0,7.50,https://cdn.myanimelist.net/images/anime/1805/...,https://myanimelist.net/anime/34134/One_Punch_...


### Modelagem

Vamos trabalhar com um filtro colaborativo para criar nosso sistema de recomendação

##### Recomendação para usuários

In [75]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy
from collections import defaultdict

def get_user_recommendations(user_name, reviews, animes, n=5):
    # Carregar os dados
    reader = Reader(rating_scale=(1, 10))
    data = Dataset.load_from_df(reviews[['review_user_name', 'anime_uid', 'review_score']], reader)

    # Dividir o conjunto de dados em treinamento e teste
    trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

    # Treinar o modelo
    model = SVD()
    model.fit(trainset)

    # Fazer recomendações para o usuário específico
    user_ratings = reviews[reviews['review_user_name'] == user_name][['anime_uid', 'review_score']]
    user_unrated_animes = reviews[~reviews['anime_uid'].isin(user_ratings['anime_uid'])]['anime_uid']

    # Criar um conjunto de gêneros favoritos do usuário
    user_favorite_genres = set(animes.loc[animes['anime_id'].isin(user_ratings['anime_uid']), 'anime_genre'].str.split(',').explode().str.strip())

    user_recommendations = set()  # Usar um conjunto para garantir valores únicos
    for anime_id in user_unrated_animes:
        anime_genres = set(animes.loc[animes['anime_id'] == anime_id, 'anime_genre'].str.split(',').explode().str.strip())
        genre_intersection = user_favorite_genres.intersection(anime_genres)

        # Atribuir uma pontuação com base na quantidade de gêneros em comum
        predicted_rating = len(genre_intersection)

        user_recommendations.add((anime_id, predicted_rating))

    # Classificar as recomendações por pontuação
    user_recommendations = sorted(user_recommendations, key=lambda x: x[1], reverse=True)

    # Retornar as recomendações
    return user_recommendations[:n]

user_name = 'treas0n'
recommendations = get_user_recommendations(user_name, reviews, animes, n=5)
print(f"Recomendações para {user_name} com base nos gêneros:")
for anime_id, score in recommendations:
    anime_info = animes.loc[animes['anime_id'] == anime_id, 'title'].values[0]
    print(f"Anime: {anime_info}, Pontuação: {score}")



Recomendações para treas0n com base nos gêneros:
Anime: One Punch Man Specials, Pontuação: 7
Anime: One Punch Man 2nd Season Commemorative Special, Pontuação: 7
Anime: One Punch Man: Road to Hero, Pontuação: 7
Anime: One Punch Man 2nd Season Specials, Pontuação: 7
Anime: Taboo Tattoo, Pontuação: 5


In [ ]:
recommendations = get_user_recommendations(user_name, reviews, animes, n=5)
print(f"Recomendações para {user_id} com base nos gêneros:")
for anime_id, score in recommendations:
    anime_info = animes.loc[animes['anime_id'] == anime_id, 'title'].values[0]
    print(f"Anime: {anime_info}, Pontuação: {score}")

In [72]:
recommendations

[(11633, 6), (32313, 5), (23317, 5), (15933, 5), (22145, 5)]

##### Recomendação para animes

In [53]:
import pandas as pd

# Suponha que anime_data é o seu DataFrame original

def get_top_users_genres(anime_data, n_top_users=5):
    # Seleciona os primeiros n_top_users usuários para cada anime
    top_users = anime_data.groupby('anime_id').head(n_top_users)

    # Agrupa por anime e concatena os favorite_genres dos usuários em uma lista
    grouped_genres = top_users.groupby('anime_id')['favorite_genres'].apply(list).reset_index()

    # Obtém os valores únicos para cada lista de favorite_genres
    grouped_genres['fav_genres_associated'] = grouped_genres['favorite_genres'].apply(lambda x: list(set(x)))

    # Merge no DataFrame original
    anime_data = pd.merge(anime_data, grouped_genres[['anime_id', 'fav_genres_associated']], on='anime_id', how='left')

    # Ajusta os valores de fav_genres_associated com base no anime_genre
    anime_data['fav_genres_associated'] = anime_data.apply(lambda row: [genre for genre in row['fav_genres_associated'] if genre in row['anime_genre']], axis=1)

    return anime_data

# Chama a função para obter a nova coluna
anime_data_fav = get_top_users_genres(anime_data)


In [54]:
anime_final = anime_data_fav.groupby(['anime_id'], as_index=False).agg({
    'title':'first',
    'anime_genre': 'first',
    'favorite_genres': 'first',
    'members': 'first',
    'score': 'first',
    'review_score': ['mean', 'count']  # Calcula a média e a contagem
})

# Corrigir os nomes das colunas
anime_final.columns = ['anime_id', 'title', 'anime_genre','favorite_genres', 'members', 'score', 'review_score_mean', 'review_score_count']

# Ordenar pelo rank
anime_final = anime_final.round({'score': 2,  'review_score_mean': 2})
anime_final = anime_final.sort_values('score', ascending=False)

anime_final.columns

Index(['anime_id', 'title', 'anime_genre', 'favorite_genres', 'members',
       'score', 'review_score_mean', 'review_score_count'],
      dtype='object')

In [56]:
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.sparse import csr_matrix, hstack

# Cria uma cópia do DataFrame original
anime_data_copy = anime_final.copy()

# Trata os gêneros como variáveis categóricas usando MultiLabelBinarizer
mlb = MultiLabelBinarizer()
genre_columns = ['anime_genre', 'favorite_genres']

# Criar DataFrames separados para anime_genre e favorite_genres
anime_genre_encoded = pd.DataFrame(mlb.fit_transform(anime_data_copy['anime_genre']), columns=mlb.classes_, index=anime_data_copy.index)
fav_genres_encoded = pd.DataFrame(mlb.transform(anime_data_copy['favorite_genres']), columns=mlb.classes_, index=anime_data_copy.index)

# Seleciona as características numéricas
numerical_cols = ['review_score_mean']

# Concatena as características numéricas com as colunas binárias dos gêneros
features_sparse = hstack([anime_genre_encoded, fav_genres_encoded, csr_matrix(anime_data_copy[numerical_cols])])

# Exibe as primeiras linhas da matriz esparsa
print(features_sparse)


  (0, 0)	1.0
  (0, 1)	1.0
  (0, 2)	1.0
  (0, 4)	1.0
  (0, 5)	1.0
  (0, 6)	1.0
  (0, 8)	1.0
  (0, 14)	1.0
  (0, 17)	1.0
  (0, 21)	1.0
  (0, 22)	1.0
  (0, 23)	1.0
  (0, 24)	1.0
  (0, 25)	1.0
  (0, 26)	1.0
  (0, 28)	1.0
  (0, 29)	1.0
  (0, 30)	1.0
  (0, 32)	1.0
  (0, 33)	1.0
  (0, 34)	1.0
  (0, 35)	1.0
  (0, 37)	1.0
  (0, 38)	1.0
  (0, 39)	1.0
  :	:
  (8088, 88)	1.6
  (8089, 88)	5.11
  (8090, 88)	6.67
  (8091, 88)	2.67
  (8092, 88)	4.5
  (8093, 88)	6.0
  (8094, 88)	5.29
  (8095, 88)	3.67
  (8096, 88)	3.0
  (8097, 88)	4.0
  (8098, 88)	3.83
  (8099, 88)	2.5
  (8100, 88)	3.22
  (8101, 88)	2.5
  (8102, 88)	3.85
  (8103, 88)	5.0
  (8104, 88)	5.83
  (8105, 88)	1.92
  (8106, 88)	5.11
  (8107, 88)	2.55
  (8108, 88)	1.42
  (8109, 88)	3.62
  (8110, 88)	3.29
  (8111, 88)	2.05
  (8112, 88)	3.22


In [57]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular a similaridade de cosseno entre os animes
anime_similarity = cosine_similarity(features_sparse)

# Função para obter as recomendações para um anime específico
def recommend_anime(anime_id, n=5):
    # Encontrar o índice do anime na matriz
    anime_index = anime_data_copy[anime_data_copy['anime_id'] == anime_id].index[0]

    # Obter as similaridades para o anime alvo
    anime_similarities = anime_similarity[anime_index]

    # Ordenar os animes por similaridade em ordem decrescente
    similar_anime_indices = anime_similarities.argsort()[::-1]

    # Excluir o anime alvo da lista de recomendações
    similar_anime_indices = similar_anime_indices[similar_anime_indices != anime_index]

    # Selecionar os top N animes recomendados
    top_recommendations = anime_data_copy.iloc[similar_anime_indices[:n]]

    return top_recommendations[['anime_id', 'title']]

# Exemplo de recomendação para um anime específico (substitua 1 pelo anime_id desejado)
anime_id_to_recommend = 269
recommendations = recommend_anime(anime_id_to_recommend, n=5)
print(recommendations)


      anime_id                                        title
6068     30015                                       ReLIFE
4953     16123                             Saki Zenkoku-hen
4474     11553                   Toradora!: Bentou no Gokui
4938     15989  Saki Achiga-hen: Episode of Side-A Specials
4312     10521                                   Working'!!


In [58]:
anime_final.loc[anime_final['title'] == 'Toradora!: Bentou no Gokui']

,anime_id,title,anime_genre,favorite_genres,members,score,review_score_mean,review_score_count
4474,11553,Toradora!: Bentou no Gokui,"['Slice of Life', 'Comedy', 'Romance', 'School']","['Comedy', 'Drama', 'Ecchi', 'Harem', 'Magic',...",110119,7.5,7.44,9


### Finalizando os datasets

Animes

In [59]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular a similaridade de cosseno entre os animes
anime_similarity = cosine_similarity(features_sparse)

# Função para obter as recomendações para um anime específico
def recommend_anime(anime_id, n=5):
    # Encontrar o índice do anime na matriz
    anime_index = anime_final[anime_final['anime_id'] == anime_id].index[0]

    # Obter as similaridades para o anime alvo
    anime_similarities = anime_similarity[anime_index]

    # Ordenar os animes por similaridade em ordem decrescente
    similar_anime_indices = anime_similarities.argsort()[::-1]

    # Excluir o anime alvo da lista de recomendações
    similar_anime_indices = similar_anime_indices[similar_anime_indices != anime_index]

    # Selecionar os top N animes recomendados
    top_recommendations = anime_final.iloc[similar_anime_indices[:n]]

    # Retornar os IDs dos animes recomendados como uma lista
    return top_recommendations['anime_id'].tolist()

# Adicionar uma coluna 'recommended_animes' ao DataFrame anime_final
anime_final['recommended_animes'] = anime_final['anime_id'].apply(lambda x: recommend_anime(x, n=5))

# Exibir as primeiras linhas do DataFrame resultante
print(anime_final[['anime_id', 'title', 'recommended_animes']])


      anime_id                               title  \
3104      5114    Fullmetal Alchemist: Brotherhood   
4015      9253                         Steins;Gate   
6515     32281                      Kimi no Na wa.   
7883     38524  Shingeki no Kyojin Season 3 Part 2   
5974     28977                            Gintama°   
...        ...                                 ...   
6053     29949                                Nami   
6375     31634                            Kokuhaku   
367        413                     Hametsu no Mars   
4709     13405                  Utsu Musume Sayuri   
2445      3287        Tenkuu Danzai Skelter+Heaven   

                      recommended_animes  
3104     [4765, 31418, 3231, 8098, 1034]  
4015      [297, 738, 33713, 10033, 9999]  
6515    [7053, 8293, 3324, 30243, 21925]  
7883  [38319, 8778, 36812, 37691, 38307]  
5974  [16247, 13469, 5177, 34104, 16592]  
...                                  ...  
6053    [3037, 2693, 36030, 33019, 3593]  
6375   

In [ ]:
# Cruzar os DataFrames pelo anime_id
df_animes_final = pd.merge(animes, anime_final[['anime_id', 'favorite_genres', 'review_score_mean', 'review_score_count', 'recommended_animes']], on='anime_id', how='left')

# Exibir as primeiras linhas do DataFrame resultante
df_animes_final.head()


,anime_id,title,synopsis,anime_genre,aired,episodes,members,popularity,rank,score,img_url,url,favorite_genres,review_score_mean,review_score_count,recommended_animes
0,1,Cowboy Bebop,"In the year 2071, humanity has colonized sever...","['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...","Apr 3, 1998 to Apr 24, 1999",26.0,930311,39,26.0,8.81,https://cdn.myanimelist.net/images/anime/4/196...,https://myanimelist.net/anime/1/Cowboy_Bebop,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",8.97,361.0,"[121, 6421, 451, 11773, 1164]"
1,5,Cowboy Bebop: Tengoku no Tobira,"Another day, another bounty—such is the life o...","['Action', 'Drama', 'Mystery', 'Sci-Fi', 'Space']","Sep 1, 2001",1.0,223199,475,149.0,8.40,https://cdn.myanimelist.net/images/anime/1439/...,https://myanimelist.net/anime/5/Cowboy_Bebop__...,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",8.26,42.0,"[30484, 2604, 32188, 35369, 1857]"
2,6,Trigun,"Vash the Stampede is the man with a $$60,000,0...","['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...","Apr 1, 1998 to Sep 30, 1998",26.0,460146,158,256.0,8.28,https://cdn.myanimelist.net/images/anime/7/203...,https://myanimelist.net/anime/6/Trigun,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",8.38,130.0,"[32287, 18045, 106, 28851, 31756]"
3,7,Witch Hunter Robin,Witches are individuals with special powers li...,"['Action', 'Magic', 'Police', 'Supernatural', ...","Jul 2, 2002 to Dec 24, 2002",26.0,85182,1278,2487.0,7.32,https://cdn.myanimelist.net/images/anime/1796/...,https://myanimelist.net/anime/7/Witch_Hunter_R...,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",7.13,30.0,"[35760, 2904, 37305, 1735, 462]"
4,8,Bouken Ou Beet,It is the dark century and the people are suff...,"['Adventure', 'Fantasy', 'Shounen', 'Supernatu...","Sep 30, 2004 to Sep 29, 2005",52.0,12319,3968,3704.0,7.02,https://cdn.myanimelist.net/images/anime/7/215...,https://myanimelist.net/anime/8/Bouken_Ou_Beet,"['Action', 'Adventure', 'Comedy', 'Mecha', 'Sc...",7.00,4.0,"[9969, 918, 36838, 37491, 15335]"


In [ ]:
df_animes_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16216 entries, 0 to 16215
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   anime_id            16216 non-null  int64  
 1   title               16216 non-null  object 
 2   synopsis            15453 non-null  object 
 3   anime_genre         16216 non-null  object 
 4   aired               16216 non-null  object 
 5   episodes            15724 non-null  float64
 6   members             16216 non-null  int64  
 7   popularity          16216 non-null  int64  
 8   rank                14553 non-null  float64
 9   score               15875 non-null  float64
 10  img_url             16051 non-null  object 
 11  url                 16216 non-null  object 
 12  favorite_genres     8113 non-null   object 
 13  review_score_mean   8113 non-null   float64
 14  review_score_count  8113 non-null   float64
 15  recommended_animes  8113 non-null   object 
dtypes: f

Users

In [79]:
'''# Lista para armazenar as recomendações de cada usuário
user_recommendations_list = []

# Iterar sobre cada usuário na tabela user_data
for user_name in user_data['user_name']:
    # Obter as recomendações para o usuário
    user_recommendations = get_user_recommendations(user_name, reviews, animes, n=5)  # Substitua pela função correta
    
    # Adicionar as recomendações à lista
    user_recommendations_list.append((user_name, user_recommendations))

# Criar um DataFrame com as recomendações
user_recommendations_df = pd.DataFrame(user_recommendations_list, columns=['user_name', 'user_recommended_animes'])

# Adicionar as recomendações à tabela users
users = pd.merge(users, user_recommendations_df, on='user_name', how='left')'''


"# Lista para armazenar as recomendações de cada usuário\nuser_recommendations_list = []\n\n# Iterar sobre cada usuário na tabela user_data\nfor user_name in user_data['user_name']:\n    # Obter as recomendações para o usuário\n    user_recommendations = get_user_recommendations(user_name, reviews, animes, n=5)  # Substitua pela função correta\n    \n    # Adicionar as recomendações à lista\n    user_recommendations_list.append((user_name, user_recommendations))\n\n# Criar um DataFrame com as recomendações\nuser_recommendations_df = pd.DataFrame(user_recommendations_list, columns=['user_name', 'user_recommended_animes'])\n\n# Adicionar as recomendações à tabela users\nusers = pd.merge(users, user_recommendations_df, on='user_name', how='left')"

In [96]:
from datetime import datetime

#users = pd.read_csv('./silver/users_cleaned.csv')
df_users_final = users.copy()

# Iterar sobre cada usuário na tabela user_data
for index, row in user_data.iterrows():
    # Obter informações do usuário
    user_name = row['user_name']
    num_reviews = row['num_reviews']
    favorite_genres = row['favorite_genres']
    
    # Adicionar informações à tabela users
    df_users_final.loc[df_users_final['user_name'] == user_name, 'num_reviews'] = num_reviews
    df_users_final.loc[df_users_final['user_name'] == user_name, 'favorite_genres'] = favorite_genres

# Suponha que 'birthday' seja a coluna que contém as datas de aniversário
df_users_final['birthday'] = pd.to_datetime(df_users_final['birthday'], errors='coerce', format='%b %d, %Y')

# Calcular a idade com base na data de nascimento e na data atual
# Calcular a idade apenas para as datas não nulas
df_users_final['age'] = df_users_final['birthday'].apply(lambda x: relativedelta(datetime.now(), x).years if not pd.isnull(x) else None)

df_users_final['num_reviews'] = df_users_final['num_reviews'].astype('int')
df_users_final['age'] = df_users_final['age'].astype('Int64')

In [111]:
df_users_final

,user_name,user_gender,birthday,favorites_anime,user_url,num_reviews,favorite_genres,age
0,-----noname-----,NaN,2019-12-31,"['6774', '245', '2001', '11061', '16592', '157...",https://myanimelist.net/profile/-----noname-----,2,"['Action', 'Sci-Fi', 'Mystery', 'Super Power']...",3
1,---SnowFlake---,NaN,NaT,"['2904', '6773', '10790']",https://myanimelist.net/profile/---SnowFlake---,1,"['Mystery', 'Police', 'Psychological', 'Supern...",<NA>
2,---was-----,NaN,NaT,[],https://myanimelist.net/profile/---was-----,1,"['Harem', 'Comedy', 'Romance', 'Ecchi', 'School']",<NA>
3,--EYEPATCH--,Male,2000-10-28,[],https://myanimelist.net/profile/--EYEPATCH--,1,"['Adventure', 'Comedy']",23
4,--Mizu--,Female,1995-07-03,"['21', '177', '6864', '4081', '5678', '23289']",https://myanimelist.net/profile/--Mizu--,2,"['Fantasy', 'Sci-Fi', 'Shounen', 'Slice of Lif...",28
...,...,...,...,...,...,...,...,...
47880,zzyamuraihazz,Male,1995-10-24,[],https://myanimelist.net/profile/zzyamuraihazz,1,"['Drama', 'Fantasy', 'Romance', 'Slice of Life']",28
47881,zzz-anime,Male,NaT,[],https://myanimelist.net/profile/zzz-anime,2,"['Action', 'Adventure', 'Fantasy', 'Harem'], [...",<NA>
47882,zzzb,Male,NaT,['9253'],https://myanimelist.net/profile/zzzb,1,['Dementia'],<NA>
47883,zzzjynne,NaN,NaT,"['31149', '1575', '2904', '43', '5337', '790',...",https://myanimelist.net/profile/zzzjynne,2,"['Action', 'Comedy', 'Super Power', 'Ecchi', '...",<NA>


In [113]:
df_animes_final.to_csv('./gold/animes_final.csv', index=False)
df_users_final.to_csv('./gold/users_final.csv', index=False)

In [ ]:
import pandas as pd
import ast

# Supondo que você já tenha os DataFrames df_animes e df_users carregados

# Inicializar listas para armazenar os dados
anime_ids = []
user_genders = []
ages = []

# Iterar pelos usuários
for _, user_row in df_users.iterrows():
    # Converter a string de lista para uma lista de Python
    favorite_animes = ast.literal_eval(user_row['favorites_anime'])
    
    # Iterar pelos animes favoritos do usuário
    for anime_id_str in favorite_animes:
        # Convertendo o anime_id para int
        anime_id = int(anime_id_str)
        
        # Verificar se o anime_id está presente no DataFrame df_animes
        if anime_id in df_animes['anime_id'].values:
            # Obter informações relevantes do anime
            anime_info = df_animes.loc[df_animes['anime_id'] == anime_id].iloc[0]
            
            # Adicionar os dados às listas
            anime_ids.append(anime_id)
            user_genders.append(user_row['user_gender'])
            ages.append(user_row['age'])

# Criar o novo DataFrame
df_distribution = pd.DataFrame({
    'anime_id': anime_ids,
    'user_gender': user_genders,
    'age': ages
})

# Exibir o novo DataFrame
df_distribution.to_csv('./gold/users_distribuition_final.csv', index=False)